<a href='http://www.holoviews.org'><img src="../notebooks/assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Excercise 4: Dynamic Interactions</h2></div>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv

hv.extension('bokeh')

In [ ]:
diamonds = pd.read_csv('../data/diamonds.csv')

### Exercise 1

As should be second nature for us now we will look at the diamonds dataframe before we start doing anything.

In [ ]:
diamonds.head()

Next we will display a static plot of 'carat' vs. 'price' as we did in the first exercise, alongside a BoxWhisker plot of the distributions.

In [ ]:
%%opts Scatter [width=600 height=400 logy=True tools=['lasso_select'] color_index='cut'] (size=1 cmap='tab20c')

scatter = hv.Scatter(diamonds.sample(10000), 'carat', ['price', 'cut', 'clarity']).select(carat=(0, 3))
boxwhisker = hv.BoxWhisker(scatter, 'clarity', 'price')

scatter + boxwhisker

Instead of a having the ``BoxWhisker`` element statically displaying the whole distribution let's instead link it to selections made on the ``Scatter`` plot. For that purpose we will declare a ``Selection1D`` stream with the ``scatter`` object as its ``source``.

The ``Selection1D`` stream provides an ``index`` of all the selected nodes. Complete the ``selection_boxwhisker`` callback to return a BoxWhisker element of the selection, plotting 'clarity' vs. 'price'. Then define a ``DynamicMap`` using the ``selection`` stream and your custom callback and lay it out next to the ``scatter`` object as above.

<b><a href="#hint1" data-toggle="collapse">Hint</a></b>

<div id="hint1" class="collapse">
A ``DynamicMap`` requires a callback function as its first argument and streams should be supplied in a list as a keyword argument.
</div>

In [ ]:
selection = hv.streams.Selection1D(source=scatter)

def selection_boxwhisker(index):
    selection = scatter.iloc[index]



<b><a href="#solution1" data-toggle="collapse">Solution</a></b>

<div id="solution1" class="collapse">
<br>
<code>selection = hv.streams.Selection1D(source=scatter)

def selection_boxwhisker(index):
    selection = scatter.iloc[index]
    return hv.BoxWhisker(selection, 'clarity', 'price')

scatter + hv.DynamicMap(selection_boxwhisker, streams=[selection])</code>
</div>

## Example 2: Streaming Data

In this example we will make use of a (simulated) streaming data source in form of taxi pickup locations. The code below splits the taxi dataset into chunks by hour which we can emit one by one emulating a live streaming data source.

In [ ]:
import time
import colorcet
from itertools import cycle
from holoviews.operation.datashader import datashade

def taxi_trips_stream(source='../data/nyc_taxi_wide.parq', frequency='H'):
    """Generate dataframes grouped by given frequency"""
    def get_group(resampler, key):
        try:
            df = resampler.get_group(key)
            df.reset_index(drop=True)
        except KeyError:
            df = pd.DataFrame()
        return df

    df = pd.read_parquet(source,
                     infer_datetime_format=True,
                     usecols=['tpep_pickup_datetime', 'pickup_x', 'pickup_y', 'total_amount'])
    df = df.set_index('tpep_pickup_datetime', drop=True)
    df = df.sort_index()
    r = df.resample(frequency)
    chunks = [get_group(r, g) for g in sorted(r.groups)]
    indices = cycle(range(len(chunks)))
    while True:
        yield chunks[next(indices)]

trips = taxi_trips_stream()
example = next(trips)

As usual let's start by inspecting the data, we have emitted an initial chunk and called it ``example``.

In [ ]:
example.head()

To begin with start by declaring a ``Pipe`` stream and initialize it with the example data. Then we define a callback to use when declaring a ``DynamicMap``. Complete the ``hourly_points`` function by returning a ``Points`` object displaying the 'pickup_x' and 'pickup_y' coordinates and the provided ``label``.

Then use that callback and the ``pipe`` stream to define a ``DynamicMap``. Finally apply the datashade operation to the DynamicMap and overlay it on top of the ``tiles``. 

**Warning**: Do not display the ``DynamicMap`` without applying the datashade operation otherwise you run the risk of freezing your browser.

<b><a href="#hint2" data-toggle="collapse">Hint</a></b>

<div id="hint2" class="collapse">
To apply datashading simply call ``datashade(dynamicmap)``.
</div>

In [ ]:
%%opts RGB [width=600 height=600]
pipe = hv.streams.Pipe(init)
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

def hourly_points(data):
    label = '%s - %s' % (str(data.index.min()), str(data.index.max()))


You should now see a map of New York City with the taxi trips on top. Run the next cell to send events to the ``Pipe`` and update the plot.

In [ ]:
for i in range(100):
    time.sleep(0.05)
    pipe.send(next(trips))

<b><a href="#solution2" data-toggle="collapse">Solution</a></b>

<div id="solution2" class="collapse">
<br>
<code>%%opts RGB [width=600 height=600]
pipe = hv.streams.Pipe(init)
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

def hourly_points(data):
    label = '%s - %s' % (str(data.index.min()), str(data.index.max()))
    return hv.Points(data, ['pickup_x', 'pickup_y'], label=label)

points = hv.DynamicMap(hourly_points, streams=[pipe])
tiles * datashade(points)</code>
</div>

## Exercise 3

In the previous exercise we used the ``Pipe`` stream, which emits just the latest chunk. Here we will stream data using the ``Buffer`` stream, which accumulates data until it's length is reached. We already defined some options, and example dataframe and the ``Buffer`` stream with a length of 1,000,000.

Start by completing the callback function so it returns a ``Curve`` plotting the 'fare_amount' against the 'tpep_pickup_datetime' and define a ``DynamicMap`` that uses this callback in combination with the ``buffer`` stream instance.

Next, apply the ``resample`` operation with ``rule='T'`` and ``function=np.sum`` and then apply the ``rolling_outlier_std`` operation to the output of that. Finally display an overlay of the ``resample`` output and the ``rolling_outlier_std`` output.

<b><a href="#hint3" data-toggle="collapse">Hint</a></b>

<div id="hint3" class="collapse">
Operations like ``resample`` and ``rolling_outlier_std`` can be chained, e.g.:
<br><br>
<code>resampled = resample(dmap)
outliers = rolling_outlier_std(resampled)
resampled * outliers
</code>
</div>

In [ ]:
%%opts Curve [width=800 height=400] (color='black' line_width=1) {+framewise} Scatter (color='red')
from holoviews.operation.timeseries import resample, rolling_outlier_std
example = next(trips)[['fare_amount']]
buffer = hv.streams.Buffer(example, length=1000000)

def fare_curve(data):
    pass



Now that you've displayed the plot, let's start sending some data to the buffer:

In [ ]:
for i in range(100):
    time.sleep(0.1)
    buffer.send(next(trips)[['fare_amount']])

<b><a href="#solution3" data-toggle="collapse">Solution</a></b>

<div id="solution3" class="collapse">
<br>
<code>%%opts Curve [width=800 height=400] (color='black' line_width=1) {+framewise} Scatter (color='red') Overlay [show_legend=False]

example = next(trips)[['fare_amount']]
buffer = hv.streams.Buffer(example, length=1000000)

def fare_curve(data):
    return hv.Curve(data, 'tpep_pickup_datetime', 'fare_amount')

fares = hv.DynamicMap(fare_curve, streams=[buffer])
minutely = resample(fares, rule='T', function=np.sum)
minutely * rolling_outlier_std(minutely, rolling_window=10)</code>
</div>